In [ ]:
from syft_rds.orchestra import setup_rds_stack
from syft_rds import RDS_NOTEBOOKS_PATH

In [ ]:
from syft_core.url import SyftBoxURL

In [ ]:
SyftBoxURL

In [ ]:
stack = setup_rds_stack(log_level="INFO")

In [ ]:
do_client = stack.do_rds_client
ds_client = stack.ds_rds_client

# Create a dataset as DO

In [ ]:
CWD = RDS_NOTEBOOKS_PATH / "quickstart"

In [ ]:
private_dir = CWD / "data" / "dataset-1" / "private"
mock_dir = CWD / "data" / "dataset-1" / "mock"
markdown_path = CWD / "data" / "dataset-1" / "description.md"

private_dir.mkdir(parents=True, exist_ok=True)
mock_dir.mkdir(parents=True, exist_ok=True)


with open(private_dir / "data.csv", "w") as f:
    f.write("-1,-2,-3")

with open(mock_dir / "data.csv", "w") as f:
    f.write("1,2,3")

with open(markdown_path, "w") as f:
    f.write("some description")

In [ ]:
data = do_client.dataset.create(
    name="dataset-1",  # MUST BE UNIQUE. Throw Exception if already exist.
    path=private_dir,  # MUST EXIST
    mock_path=mock_dir,
    summary="dummy data",
    description_path=markdown_path,
)

In [ ]:
do_client.datasets

# Use dataset as DS

In [ ]:
ds_client.datasets

In [ ]:
ds_client.dataset.get(name="dataset-1").get_mock_path()

In [ ]:
ds_client.dataset.get(name="dataset-1").get_private_path()

In [ ]:
dataset = ds_client.dataset.get(name="dataset-1")
dataset.describe()

In [ ]:
my_file = CWD / "data" / "code1.py"

In [ ]:
%%writefile {my_file}

import os

DATA_DIR = os.environ["DATA_DIR"]
OUTPUT_DIR = os.environ["OUTPUT_DIR"]

print("Hello, world!")

with open(os.path.join(OUTPUT_DIR, "output.txt"), "w") as f:
    f.write("ABC")


# Submit a job to create UserCode and a Job

In [ ]:
job = ds_client.jobs.submit(user_code_path=my_file, dataset_name=dataset.name)
job

In [ ]:
# Code file is moved to a location inside syftbox

code = ds_client.user_code.get(uid=job.user_code_id)
print(code)

print(code.local_file)

assert code.local_file.exists()

In [ ]:
jobs = do_client.jobs.get_all()

In [ ]:
job = jobs[-1]

In [ ]:
job

In [ ]:
res_job = do_client.run_private(job)

In [ ]:
do_client.jobs.share_results(res_job);

In [ ]:
# job.add_to_queue()

In [ ]:
job = ds_client.jobs.get_all()[-1]

In [ ]:
job._syftbox_client

In [ ]:
ds_client._syftbox_client.datasites

In [ ]:
job.output_url

In [ ]:
job.get_output_path()